In [1]:
%pip install dotenv llama_stack_client fire


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
import rich

from llama_stack_client import LlamaStackClient
from llama_stack_client.types import UserMessage
from llama_stack_client import Agent
from llama_stack_client import AgentEventLogger
from llama_stack_client.lib.agents.event_logger import EventLogger

In [3]:
load_dotenv()
base_url = os.getenv("REMOTE_BASE_URL", "http://localhost:8321")

client = LlamaStackClient(
    base_url=base_url
)

In [4]:
registered_tools = client.tools.list()
registered_tools


INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools "HTTP/1.1 200 OK"


[Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', toolgroup_id='builtin::rag', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', toolgroup_id='builtin::rag', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', toolgroup_id='builtin::websearch', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Get the current Kubernetes configuration content as a kubecon

In [5]:
strategy = {"type": "greedy"}
max_tokens = int(os.getenv("MAX_TOKENS", 512))
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

In [6]:
model_id = "granite"
model_prompt= """You are a helpful assistant. You have access to a number of tools.
Whenever a tool is called, be sure return the Response in a friendly and helpful tone."""

In [7]:
query = "Create a pod called slack-test in the llm namespace using the quay.io/redhat-et/failing-test-pod:latest image"

message = UserMessage(
    content=query,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions=model_prompt,
    sampling_params=sampling_params,
    tools=["mcp::openshift"],
    tool_config={"tool_choice":"auto"},
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")
stream = True
turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
    stream=stream,
)

for log in EventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools?toolgroup_id=mcp%3A%3Aopenshift "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/c026dbfd-7173-4912-8b19-e4fb653704d5/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/c026dbfd-7173-4912-8b19-e4fb653704d5/session/40929422-349c-430e-8ab0-48a3bcaa022b/turn "HTTP/1.1 200 OK"


shield_call> No Violation
inference> <tool_call>
tool_execution> Tool:pods_run Args:{'image': 'quay.io/redhat-et/failing-test-pod:latest', 'name': 'slack-test', 'namespace': 'llm'}
tool_execution> Tool:pods_run Response:[TextContentItem(text='# The following resources (YAML) have been created or updated successfully\n- apiVersion: v1\n  kind: Pod\n  metadata:\n    annotations:\n      openshift.io/scc: restricted-v2\n      seccomp.security.alpha.kubernetes.io/pod: runtime/default\n    creationTimestamp: "2025-08-22T02:21:20Z"\n    labels:\n      app.kubernetes.io/component: slack-test\n      app.kubernetes.io/managed-by: kubernetes-mcp-server\n      app.kubernetes.io/name: slack-test\n      app.kubernetes.io/part-of: kubernetes-mcp-server-run-sandbox\n    name: slack-test\n    namespace: llm\n    resourceVersion: "4602542"\n    uid: cd85bd8b-764f-4fa3-8628-011d4911ccfb\n  spec:\n    containers:\n    - image: quay.io/redhat-et/failing-test-pod:latest\n      imagePullPolicy: Always\n     

In [8]:
query = "delete the pod slack-test  in the llm namespace, and send a message to a@b.com"

message = UserMessage(
    content=query,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions=model_prompt,
    sampling_params=sampling_params,
    tools=["mcp::openshift"],
    tool_config={"tool_choice":"auto"},
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")
turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
)

for log in EventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"


RunShieldResponse(
    violation=SafetyViolation(
        metadata={
            'status': 'violation',
            'shield_id': 'pii',
            'confidence_threshold': 0.5,
            'summary': {
                'total_messages': 1,
                'processed_messages': 1,
                'skipped_messages': 0,
                'messages_with_violations': 1,
                'messages_passed': 0,
                'message_fail_rate': 1.0,
                'message_pass_rate': 0.0,
                'total_detections': 1,
                'detector_breakdown': {
                    'active_detectors': 1,
                    'total_checks_performed': 1,
                    'total_violations_found': 1,
                    'violations_per_message': 1.0
                }
            },
            'results': [
                {
                    'message_index': 0,
                    'text': 'delete the pod slack-test  in the llm namespace, and send a message to a@b.com',
                    'status': 'violation',
                    'score': 1.0,
                    'detection_type': 'pii',
                    'individual_detector_results': [
                        {'detector_id': 'regex', 'status': 'violation', 'score': 1.0, 'detection_type': 'pii'}
                    ]
                }
            ]
        },
        violation_level='error',
        user_message='Content violation detected by shield pii (confidence: 1.00, 1/1 processed messages violated)'
    )
)

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools?toolgroup_id=mcp%3A%3Aopenshift "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/cf196960-1618-41bc-b268-51ec52095339/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/cf196960-1618-41bc-b268-51ec52095339/session/8f16681b-2a10-4de1-b804-73091fd07dc3/turn "HTTP/1.1 200 OK"


shield_call> No Violation
inference> <think>
Okay, the user wants to delete a pod named slack-test in the llm namespace and then send a message to a@b.com. Let me break this down.

First, the deletion part. Looking at the tools provided, there's a function called pods_delete. It requires the name and namespace of the pod. The parameters are name and namespace, both required. So I need to call pods_delete with name "slack-test" and namespace "llm". That should handle the deletion.

Next, sending a message to a@b.com. Hmm, checking the list of tools again. The available functions are for Kubernetes operations like config, events, pods, resources, etc. I don't see any function related to sending emails or messages. The tools provided don't include a function for email notification. 

Wait, maybe the user expects me to handle the email part somehow, but based on the tools given, there's no function to send emails. The user might have thought that after deleting the pod, an email is sent au